# Tutorial: Moving from Single Jobs to Many Nodes: Dask, X-Array, and Pangeo, Part 1

This is the first of a two notebook series which introduces the reader to basic concepts related to moving basic xarray workflows from single-machine to many-machine systems. This material is adapted from the excellent tutorial developed by [Ryan Abernathey, Joe Hamman, and Scott Henderson from the AGU 2018 Fall Meeting](https://github.com/pangeo-data/pangeo-tutorial-agu-2018/).

# Xarray for multidimensional labeled data

Xarray is designed to make it easier to work with with _labeled multidimensional data_. By _multidimensional data_ (also often called _N-dimensional_), we mean data with many independent dimensions or axes. For example, we might represent Earth's surface temperature $T$ as a three dimensional variable

$$ T(x, y, t) $$

where $x$ and $y$ are spatial dimensions and and $t$ is time. By _labeled_, we mean data that has metadata associated with it describing the names and relationships between the variables. The cartoon below shows a "data cube" schematic dataset with temperature and preciptation sharing the same three dimensions, plus longitude and latitude as auxilliary coordinates.

![xarray data model](https://github.com/pydata/xarray/raw/master/doc/_static/dataset-diagram.png)

--- 

Initial setup matter

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.style.use(['seaborn-ticks', 'seaborn-talk'])

import numpy as np
import xarray as xr

## Sample geospatial data

One of the major applications that `xarray` is used for within the geosciences community is for interacting with gridded datasets, such as those common in the atmospheric, oceanographic, and climate sciences. Often times, analyses, model outputs or other datasets are distributed in a binary format such as NetCDF; xarray's core data structures can natively read and understand these files, and provides simple tools for manipulating their underlying data.

To illustrate this, we'll go ahead and load a NOAA ERSST sea surface temperature analysis dataset, which has been pre-downloaded onto this Binder machine.

In [ ]:
ds = xr.open_dataset('../data/sst/NOAA_NCDC_ERSST_v3b_SST-1960.nc')
ds

### Datasets

When we load this file, we're retuend a special data structure called a `Dataset`. What is a dataset? A Dataset is simply an object wihich holds many DataArrays which potentially can share coordinates.

Datasets have three main elements:
- Data variables
- Coordiantes (which provide labels and other ancillary information about the data variables)
- Attributes (additional metadata for the dataset)

This dataset only contains one variable (`sst`), which we can access in the following two ways

In [ ]:
# both do the exact same thing

# dictionary syntax
sst = ds['sst']

# attribute syntax
sst = ds.sst

sst

### Multidimensional Indexing

In this example, we take advantage of the fact that xarray understands time to select a particular date

In [ ]:
sst.sel(time='1960-06-15').plot(vmin=-2, vmax=30)

But we can select along any axis

In [ ]:
sst.sel(lon=180).transpose().plot()

In [ ]:
sst.sel(lon=180, lat=40).plot()

### Label-Based Reduction Operations

Usually the process of data analysis involves going from a big, multidimensional dataset to a few concise figures.
Inevitably, the data must be "reduced" somehow. Examples of simple reduction operations include:

- Mean
- Standard Deviation
- Minimum
- Maximum

etc. Xarray supports all of these and more, via a familiar numpy-like syntax. But with xarray, you can specify the reductions by dimension.

First we start with the default, reduction over all dimensions:

In [ ]:
sst.mean()

In [ ]:
sst_time_mean = sst.mean(dim='time')
sst_time_mean.plot(vmin=-2, vmax=30)

In [ ]:
sst_zonal_mean = sst.mean(dim='lon')
sst_zonal_mean.transpose().plot()

In [ ]:
sst_time_and_zonal_mean = sst.mean(dim=('time', 'lon'))
sst_time_and_zonal_mean.plot()

In [ ]:
# some might prefer to have lat on the y axis
sst_time_and_zonal_mean.plot(y='lat')

### More Complicated Example: Weighted Mean

The means we calculated above were "naive"; they were straightforward numerical means over the different dimensions of the dataset. They did not account, for example, for spherical geometry of the globe and the necessary weighting factors. Although xarray is very useful for geospatial analysis, **it has no built-in understanding of geography**.

Below we show how to create a proper weighted mean by using the formula for the area element in spherical coordinates. This is a good illustration of several xarray concepts.

The [area element for lat-lon coordinates](https://en.wikipedia.org/wiki/Spherical_coordinate_system#Integration_and_differentiation_in_spherical_coordinates) is

$$ \delta A = R^2 \delta \phi \delta \lambda \cos(\phi) $$

where $\phi$ is latitude, $\delta \phi$ is the spacing of the points in latitude, $\delta \lambda$ is the spacing of the points in longitude, and $R$ is Earth's radius. (In this formula, $\phi$ and $\lambda$ are measured in radians.) Let's use xarray to create the weight factor.

In [ ]:
R = 6.37e6
# we know already that the spacing of the points is one degree latitude
dϕ = np.deg2rad(1.)
dλ = np.deg2rad(1.)
dA = R**2 * dϕ * dλ * np.cos(np.deg2rad(ds.lat))
dA.plot()

In [ ]:
dA.where(sst[0].notnull())

In [ ]:
pixel_area = dA.where(sst[0].notnull())
pixel_area.plot()

In [ ]:
total_ocean_area = pixel_area.sum(dim=('lon', 'lat'))
sst_weighted_mean = (sst * pixel_area).sum(dim=('lon', 'lat')) / total_ocean_area
sst_weighted_mean.plot()

### Maps

Xarray integrates with cartopy to enable you to plot your data on a map

In [ ]:
import cartopy.crs as ccrs

plt.figure(figsize=(12, 8))
ax = plt.axes(projection=ccrs.InterruptedGoodeHomolosine())
ax.coastlines()

sst[0].plot(transform=ccrs.PlateCarree(), vmin=-2, vmax=30,
            cbar_kwargs={'shrink': 0.4})

## Opening Multi-file Datastores

One of the most useful features of `xarray` is its ability to open datasets which are split or "chunked" across many files. This is very common in atmospheric/climate science, particularly because numerical models typically write out many fields but for a single time step. So what is often the case is that you may wish to read a small portion of a very large datastore, distributed across many files on disk.

`xarray` provides a simple function, **open_mfdataset()**, which helps automate this very common task.

In [ ]:
ds_all = xr.open_mfdataset('../data/sst/*.nc').load()
ds_all

Now we have 57 years of data instead of one!

## Groupby

Now that we have a bigger dataset, this is a good time to check out xarray's groupby capabilities.

In [ ]:
sst_clim = ds_all.sst.groupby('time.month').mean(dim='time')
sst_clim

Now the data has dimension `month` instead of time!
Each value represents the average among all of the Januaries, Februaries, etc. in the dataset.

In [ ]:
(sst_clim[6] - sst_clim[0]).plot()
plt.title('June minus July SST Climatology')

## Resample and Rolling

Resample is meant specifically to work with time data (data with a `datetime64` variable as a dimension).
It allows you to change the time-sampling frequency of your data.

Let's illustrate by selecting a single point.

In [ ]:
sst_ts = ds_all.sst.sel(lon=300, lat=10)
sst_ts_annual = sst_ts.resample(time='A').mean(dim='time')
sst_ts_annual

In [ ]:
sst_ts.plot()
sst_ts_annual.plot()

An alternative approach is a "running mean" over the time dimension.
This can be accomplished with xarray's `.rolling` operation.

In [ ]:
sst_ts_rolling = sst_ts.rolling(time=24).mean(dim='time', centered=True)
sst_ts_annual.plot(marker='o')
sst_ts_rolling.plot()